In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../datageneration')
from heatconduction2d import main, postprocessdata, generatedataset
from randompolynomials import randompoly1DO3, randompoly2DO3

In [7]:
simparams = {'inputdata': 'poly',
             'nelems': 32,
             'etype': 'square',
             'btype': 'spline',
             'basisdegree': 1,
             'intdegree': 2,
             'nfemsamples': 2}

trainingdataparams = {'N_sensornodes': 144,
                      'N_outputnodes': 100,
                      'N_sims': 3}

params = {}
params['simparams'] = simparams
params['trainingdataparams'] = trainingdataparams

c_theta = np.random.uniform(-0.1, 0.1, 10)
c_f = np.random.uniform(-0.1, 0.1, 10)
c_etab = np.random.uniform(-0.1, 0.1, 4)
c_etat = np.random.uniform(-0.1, 0.1, 4)

theta = randompoly2DO3(c_theta)
f = randompoly2DO3(c_f)
etab = randompoly1DO3(c_etab)
etat = randompoly1DO3(c_etat)
gl = 0
gr = 0

inputs = {'theta': theta,
          'f': f,
          'etab': etab,
          'etat': etat,
          'gl': gl,
          'gr': gr}

outputs = main(params, inputs, save=False, savedir='../../../trainingdata', label='test')

# generatedataset(params, save=True, savedir='../../../trainingdata', label='test')

optimize > constrained 66/1089 dofs
optimize > optimum value 0.00e+00
solve > solving 1023 dof system to machine precision using arnoldi solver
solve > solver returned with residual 9e-17
Simulation: 0
optimize > constrained 66/1089 dofs
optimize > optimum value 0.00e+00
solve > solving 1023 dof system to machine precision using arnoldi solver
solve > solver returned with residual 2e-16
Simulation: 1
optimize > constrained 66/1089 dofs
optimize > optimum value 0.00e+00
solve > solving 1023 dof system to machine precision using arnoldi solver
solve > solver returned with residual 2e-16
Simulation: 2
optimize > constrained 66/1089 dofs
optimize > optimum value 0.00e+00
solve > solving 1023 dof system to machine precision using arnoldi solver
solve > solver returned with residual 2e-16


In [ ]:
data_postprocessed = postprocessdata(params, inputs, outputs)
data_postprocessed['x'].shape

In [ ]:
generatedataset(params, save, savedir, label)

In [ ]:
x = outputs['x']
u = outputs['u']

#plot
fig, axs = plt.subplots(1,3, figsize=(16, 4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.2)
size_title = 25
size_ticklabels = 15
size_axeslabels = 20

ax = axs[0]
plot0 = ax.tripcolor(x[:,0], x[:,1], theta(x[:,0],x[:,1]))
ax.set_title(r"$\theta(\mathbf{x})$", fontsize=size_title)
ax.tick_params(axis='both', labelsize=size_ticklabels)
ax.ticklabel_format(style='sci', scilimits=[-3,3])
ax.set_xlabel(r'$x$', fontsize=size_axeslabels)
ax.set_ylabel(r'$y$', fontsize=size_axeslabels)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_aspect('equal')
fig.colorbar(plot0, ax=ax)

ax = axs[1]
plot1 = ax.tripcolor(x[:,0], x[:,1], f(x[:,0],x[:,1]))
ax.set_title(r"$f(\mathbf{x})$", fontsize=size_title)
ax.tick_params(axis='both', labelsize=size_ticklabels)
ax.ticklabel_format(style='sci', scilimits=[-3,3])
ax.set_xlabel(r'$x$', fontsize=size_axeslabels)
ax.set_ylabel(r'$y$', fontsize=size_axeslabels)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_aspect('equal')
fig.colorbar(plot1, ax=ax)

ax = axs[2]
ax.plot(x[:,0], etab(x[:,0]), label=r'$\eta_\mathrm{bottom}(x)$')
ax.plot(x[:,0], etat(x[:,0]), label=r'$\eta_\mathrm{top}(x)$')
ax.set_title(r"$\eta(\mathbf{x})$", fontsize=size_title)
ax.tick_params(axis='both', labelsize=size_ticklabels)
ax.ticklabel_format(style='sci', scilimits=[-3,3])
ax.set_xlabel(r'$x$', fontsize=size_axeslabels)
ax.set_ylabel(r'$\eta(x)$', fontsize=size_axeslabels)
ax.set_xlim(0,1)
ax.legend(fontsize=15)
ax.set_box_aspect(1)

# plt.savefig("heatconduction_theta_f_eta.svg", bbox_inches='tight')

fig, ax = plt.subplots(1,1, figsize=(6, 4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.4, hspace=0.4)

fig = ax.tripcolor(x[:,0], x[:,1], u)
ax.set_title(r"$u(\mathbf{x})$", fontsize=size_title)
ax.tick_params(axis='both', labelsize=size_ticklabels)
ax.ticklabel_format(style='sci', scilimits=[-3,3])
ax.set_xlabel(r'$x$', fontsize=size_axeslabels)
ax.set_ylabel(r'$y$', fontsize=size_axeslabels)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_aspect('equal')
plt.colorbar(fig)

In [ ]:
X, Y = np.mgrid[0:1:12j, 0:1:12j]
positions = np.vstack([X.ravel(), Y.ravel()])
plt.scatter(positions[0],positions[1], c=f(positions[0],positions[1]))
plt.axis('equal')

In [ ]:
plt.matshow(np.rot90(f(positions[0],positions[1]).reshape(12,12)))

In [ ]:
plt.scatter(x[:,0],x[:,1])

In [ ]:
N_sensornodes = 144
N_outputnodes = 100

x_sensor, y_sensor = np.mgrid[0:1:np.sqrt(N_sensornodes)*1j, 0:1:np.sqrt(N_sensornodes)*1j]
sensornodes_Omega = np.vstack([x_sensor.ravel(), y_sensor.ravel()]).T
theta_sensor = theta(sensornodes_Omega[:,0], sensornodes_Omega[:,1])
f_sensor  = f(sensornodes[:,0], sensornodes[:,1])

sensornodes_Gamma = np.linspace(0,1,int(N_sensornodes/2))
etab_sensor = etab(sensornodes_Gamma)
etat_sensor = etat(sensornodes_Gamma)
eta_sensor = np.concatenate((etab_sampled, etat_sampled))

indices = np.linspace(0,x.shape[0]-1, x.shape[0], dtype=int)
indices_output = np.random.choice(indices, size=N_outputnodes, replace=False)
x_output = x[indices_output]
u_output = u[indices_output]

theta_post = np.tile(theta_sensor, (N_outputnodes, 1))
f_post = np.tile(f_sensor, (N_outputnodes, 1))
eta_post = np.tile(eta_sensor, (N_outputnodes, 1))

In [ ]:
plt.scatter(x[:,0],x[:,1], c=u)

In [ ]:
plt.scatter(x_output[:,0],x_output[:,1], c=u_output)